In [34]:
from propositional import *
from pprint import pprint

In [35]:
# Construct an Ann object from a string.
s = " imp  intro   4-8 "
try:
  ann = Ann(s)
  print(f"input_str = '{ann.input_str}'")
  print(ann.build_str())
  print("\noutput from __str__():")
  print(" ", ann)
except Exception as e:
    print(e)  

input_str = ' imp  intro   4-8 '
rule: imp intro
premise: [(4, 8)]

output from __str__():
  imp intro 4-8


In [36]:
# Construct a NodeLabel object from a string.
line = "A imp B .hyp"
label = NodeLabel('formula', line)
print(label.build_str())
print()
print(label)

type: formula
line: A imp B .hyp
formula: A imp B
ann: hyp
is_hyp: True

A imp B	 .hyp


In [37]:
# Construct a ProofNode manually. (not by using the parser)
line1 = "A imp B .hyp"
label1 = NodeLabel('formula', line1)
node1 = ProofNode(label1)
line2 = "A .hyp"
label2 = NodeLabel('formula', line2)
node2 = ProofNode(label2)
line3 = "B .imp elim 1,2"
label3 = NodeLabel('formula', line3)
node3 = ProofNode(label3)
line4 = "A .repeat 2"
label4 = NodeLabel('formula', line4)
node4 = ProofNode(label4)
label0 = NodeLabel()
node0 = ProofNode(label0, [node1, node2, node3, node4])
node0.build_index()
print(node0)

│1. A imp B	 .hyp
│2. A	 .hyp
├─
│3. B	 .imp elim 1,2
│4. A	 .repeat 2



In [38]:
# Another example, similar to the one above.
line1 = "A imp B .hyp"
label1 = NodeLabel('formula', line1)
node1 = ProofNode(label1)
line2 = "B imp C .hyp"
label2 = NodeLabel('formula', line2)
node2 = ProofNode(label2)
line3 = "A .hyp"
label3 = NodeLabel('formula', line3)
node3 = ProofNode(label3)
line4 = "B .imp elim 1,3"
label4 = NodeLabel('formula', line4)
node4 = ProofNode(label4)
line5 = "C .imp elim 2,4"
label5 = NodeLabel('formula', line5)
node5 = ProofNode(label5)
line6 = "A imp C .imp intro 3-5"
label6 = NodeLabel('formula', line6)
node6 = ProofNode(label6)
label_root = NodeLabel()
label_sub = NodeLabel()
node_sub = ProofNode(label_sub, [node3, node4, node5])
node_root = ProofNode(label_root, [node1, node2, node_sub, node6])
node_root.build_index()
print(node_root)

│1. A imp B	 .hyp
│2. B imp C	 .hyp
├─
││3. A	 .hyp
│├─
││4. B	 .imp elim 1,3
││5. C	 .imp elim 2,4
│6. A imp C	 .imp intro 3-5



In [39]:
# This is a test for the preprocessing function get_str_li().
# The output of get_str_li() is a list of strings 
# to be fed to the parser.

# You may or may not prefix each line with line number.
# The parser can assign the correct line number 
# to each node anyway.
# You may also use | instead  of TAB to indent subproofs.
# The following 4 input strings are all equivalent---they 
# are parsed into the same proof tree.

prf_str1 = '''
1. A imp B .hyp
2. B imp C .hyp
proves
  3. A .hyp
  proves
  4. B     .imp elim 1,3
  5. C     .imp elim 2,4
6. A imp C .imp intro 3-5
'''
# or,
prf_str2 = '''
│1. A imp B	 .hyp
│2. B imp C	 .hyp
├─
││3. A	 .hyp
│├─
││4. B	 .imp elim 1,3
││5. C	 .imp elim 2,4
│6. A imp C	 .imp intro 3-5
'''
# or, above two without numbering
prf_str3 = '''
A imp B .hyp
B imp C .hyp
proves
  A .hyp
  proves
  B     .imp elim 1,3
  C     .imp elim 2,4
A imp C .imp intro 3-5
'''
# or,
prf_str4 = '''
│A imp B	 .hyp
│B imp C	 .hyp
├─
││A	 .hyp
│├─
││B	 .imp elim 1,3
││C	 .imp elim 2,4
│A imp C	 .imp intro 3-5
'''
# Convert either string to the following.
'''
A imp B	 .hyp
B imp C	 .hyp
{{
  A	 .hyp
  B	 .imp elim 1,3
  C	 .imp elim 2,4
}}
A imp C	 .imp intro 3-5
'''

for i in range(1, 5):
  prf_str_name = f"prf_str{i}"
  prf_str = eval(prf_str_name)
  print(f"[{prf_str_name}]:")
  lines = get_str_li(prf_str, tabsize=2)
  print_lines(lines)    
  print()

[prf_str1]:
A imp B .hyp
B imp C .hyp
{{
  A .hyp
  B     .imp elim 1,3
  C     .imp elim 2,4
}}
A imp C .imp intro 3-5

[prf_str2]:
A imp B	 .hyp
B imp C	 .hyp
{{
  A	 .hyp
  B	 .imp elim 1,3
  C	 .imp elim 2,4
}}
A imp C	 .imp intro 3-5

[prf_str3]:
A imp B .hyp
B imp C .hyp
{{
  A .hyp
  B     .imp elim 1,3
  C     .imp elim 2,4
}}
A imp C .imp intro 3-5

[prf_str4]:
A imp B	 .hyp
B imp C	 .hyp
{{
  A	 .hyp
  B	 .imp elim 1,3
  C	 .imp elim 2,4
}}
A imp C	 .imp intro 3-5



In [40]:
# Another example.

# This is a Fitch proof for 
#         A -> B |- (A or C) -> (B or C)
#
prf_str = '''
1. A imp B .hyp
proves
  2. A or C .hyp
  proves
    3. A .hyp   
    proves
    4. B .imp elim 1,3
    5. B or C .or intro 4
    6. C .hyp
    proves
    7. B or C .or intro 6
  8. B or C .or elim 2,3-5,6-7
9. A or C imp B or C .imp intro 2-8'''
lines = get_str_li(prf_str, tabsize=2)
print_lines(lines)


A imp B .hyp
{{
  A or C .hyp
  {{
    A .hyp   
    B .imp elim 1,3
    B or C .or intro 4
  }}
  {{
    C .hyp
    B or C .or intro 6
  }}
  B or C .or elim 2,3-5,6-7
}}
A or C imp B or C .imp intro 2-8


In [41]:
# This is a test for the parser.
prf_str = '''
1. A .hyp
2. B  .hyp
proves
3. A and B .and intro
'''
proof = parse_fitch(prf_str)
print(proof)

│1. A	 .hyp
│2. B	 .hyp
├─
│3. A and B	 .and intro



In [42]:
# Another test for the parser.
prf_str = '''
1. A imp B .hyp
2. B imp C .hyp
proves
  3. A .hyp
  proves
  4. B     .imp elim 1,3
  5. C     .imp elim 2,4
6. A imp C .imp intro 3-5
'''
print(proof := parse_fitch(prf_str))

│1. A imp B	 .hyp
│2. B imp C	 .hyp
├─
││3. A	 .hyp
│├─
││4. B	 .imp elim 1,3
││5. C	 .imp elim 2,4
│6. A imp C	 .imp intro 3-5



In [43]:
# Another test for the parser.
# Note that the parser can handle the case where
# two subproofs at the same level are adjacent.
# It is a little hard to see that line 6 starts a new subproof.
# This will be solved by the LaTeX output later.
prf_str = '''
1. A imp B .hyp
proves
  2. A or C .hyp
  proves
    3. A .hyp
    proves
    4. B .imp elim 1,3
    5. B or C .or intro 4
    6. C .hyp
    proves
    7. B or C .or intro 6
  8. B or C .or elim 2,3-5,6-7
9. A or C imp B or C .imp intro 2-8
'''
print(proof := parse_fitch(prf_str))

│1. A imp B	 .hyp
├─
││2. A or C	 .hyp
│├─
│││3. A	 .hyp
││├─
│││4. B	 .imp elim 1,3
│││5. B or C	 .or intro 4
│││6. C	 .hyp
││├─
│││7. B or C	 .or intro 6
││8. B or C	 .or elim 2,3-5,6-7
│9. A or C imp B or C	 .imp intro 2-8



In [44]:
index_dict = proof.build_index_dict()
pprint(index_dict, sort_dicts=False)

{'1-9': [0],
 '1': [0, 0],
 '2-8': [0, 1],
 '2': [0, 1, 0],
 '3-5': [0, 1, 1],
 '3': [0, 1, 1, 0],
 '4': [0, 1, 1, 1],
 '5': [0, 1, 1, 2],
 '6-7': [0, 1, 2],
 '6': [0, 1, 2, 0],
 '7': [0, 1, 2, 1],
 '8': [0, 1, 3],
 '9': [0, 2]}


In [45]:
for i in index_dict:
  for j in index_dict:
    label_j = proof.get_node(j).label
    if label_j.type == 'formula' and not label_j.is_hyp: # conclusion
      if proof.is_earlier(i, j):
        print(f"{i} <= {j}") 


1 <= 4
1 <= 5
1 <= 7
1 <= 8
1 <= 9
2-8 <= 9
2 <= 4
2 <= 5
2 <= 7
2 <= 8
3-5 <= 7
3-5 <= 8
3 <= 4
3 <= 5
4 <= 5
6-7 <= 8
6 <= 7


In [46]:
fmla = FormulaProp('bot')
node1 = parse_ast('not (A iff not C_1)')
node2 = parse_ast('A iff not C_1')
fmla.verified_by(RuleInfer.BOT_INTRO, [node1, node2])

True

In [47]:
fmla.verified_by(RuleInfer.BOT_INTRO, [node2, node1])

True

In [48]:
node1 = parse_ast('bot')
fmla = FormulaProp('not (A iff not C_1)')
fmla.verified_by(RuleInfer.BOT_ELIM, [node1])

True

In [49]:
node1 = parse_ast('A imp bot')
fmla = FormulaProp('not A')
fmla.verified_by(RuleInfer.NOT_INTRO, [node1])

True

In [50]:
node1 = parse_ast('not A imp bot')
fmla = FormulaProp('A')
fmla.verified_by(RuleInfer.NOT_ELIM, [node1])

True

In [51]:
node1 = parse_ast('A imp B')
node2 = parse_ast('forall x B1(x)')
fmla = FormulaProp('forall x B1(x) and (A imp B)')
fmla.verified_by(RuleInfer.AND_INTRO, [node1, node2])

True

In [52]:
node2 = parse_ast('A imp B')
node1 = parse_ast('forall x B1(x)')
fmla = FormulaProp('forall x B1(x) and (A imp B)')
fmla.verified_by(RuleInfer.AND_INTRO, [node1, node2])

True

In [53]:
node1 = parse_ast('forall x B1(x) and (A imp B)')
fmla1 = FormulaProp('A imp B')
fmla2 = FormulaProp('forall x B1(x)')
print(fmla1.verified_by(RuleInfer.AND_ELIM, [node1]))
print(fmla2.verified_by(RuleInfer.AND_ELIM, [node1]))

True
True


In [54]:
node1 = parse_ast('A')
node2 = parse_ast('B')
fmla = FormulaProp('A or B')
print(fmla.verified_by(RuleInfer.OR_INTRO, [node1]))
print(fmla.verified_by(RuleInfer.OR_INTRO, [node2]))


True
True


In [55]:
node1 = parse_ast('A or B')
node2 = parse_ast('A imp C')
node3 = parse_ast('B imp C')
fmla = FormulaProp('C')
print(fmla.verified_by(RuleInfer.OR_ELIM, [node1, node2, node3]))
print(fmla.verified_by(RuleInfer.OR_ELIM, [node1, node3, node2]))
print(fmla.verified_by(RuleInfer.OR_ELIM, [node2, node1, node3]))
print(fmla.verified_by(RuleInfer.OR_ELIM, [node2, node3, node1]))
print(fmla.verified_by(RuleInfer.OR_ELIM, [node3, node1, node2]))
print(fmla.verified_by(RuleInfer.OR_ELIM, [node3, node2, node1]))


True
True
True
True
True
True


In [56]:
node1 = parse_ast('A imp B')
fmla = FormulaProp('A imp B')
print(fmla.verified_by(RuleInfer.IMP_INTRO, [node1]))

True


In [64]:
fmla = FormulaProp('B1(x)')
node1 = parse_ast('(A iff not C_1) imp B1(x)')
node2 = parse_ast('A iff not C_1')
print(fmla.verified_by(RuleInfer.IMP_ELIM, [node1, node2]))
print(fmla.verified_by(RuleInfer.IMP_ELIM, [node2, node1]))
print(fmla.verified_by(RuleInfer.AND_INTRO, [node1, node2]))

True
True
False


In [58]:
fmla = FormulaProp('not C_1 iff B1(x)')
node1 = parse_ast('not C_1 imp B1(x)')
node2 = parse_ast('B1(x) imp not C_1')
print(fmla.verified_by(RuleInfer.IFF_INTRO, [node1, node2]))

True


In [59]:
fmla = FormulaProp('B1(x)')
node2 = parse_ast('not C_1 iff B1(x)')
node1 = parse_ast('not C_1')
print(fmla.verified_by(RuleInfer.IFF_ELIM, [node1, node2]))


True


In [60]:
fmla = FormulaProp('not C_1')
node2 = parse_ast('not C_1 iff B1(x)')
node1 = parse_ast('B1(x)')
print(fmla.verified_by(RuleInfer.IFF_ELIM, [node1, node2]))


True


In [61]:
node1 = parse_ast('A or B')
fmla = FormulaProp('A or B')
print(fmla.verified_by(RuleInfer.REPEAT, [node1]))

True


In [62]:
fmla1 = FormulaProp('not not A_1 or not A_1')
fmla2 = FormulaProp('A_1')
print(fmla1.verified_by(RuleInfer.LEM))
print(fmla2.verified_by(RuleInfer.LEM))
print(fmla1.verified_by(RuleInfer.HYP))
print(fmla1.verified_by(RuleInfer.HYP))

True
False
True
True
